# **This notebook aims to load data from [cmems data store](https://data.marine.copernicus.eu/products) and adapt it to the pangeo-fish format**

## **1. Copernicus actual data format, from the following [notebook](https://github.com/IAOCEA/pangeo-fish/blob/main/notebooks/pangeo-fish.ipynb)**

In [1]:
# Import necessary libraries and modules.
import xarray as xr
from pint_xarray import unit_registry as ureg
from pangeo_fish.io import open_tag
import hvplot.xarray

In [3]:
# catalog_url specifies the URL for the catalog for reference data used.
catalog_url = "https://data-taos.ifremer.fr/kerchunk/ref-copernicus.yaml"

# bbox, bounding box, defines the latitude and longitude range for the analysis area.


# relative_depth_threshold defines the acceptable fish depth relative to the maximum tag depth.
# It determines whether the fish can be considered to be in a certain location based on depth.
relative_depth_threshold = 0.8

In [4]:
# Import necessary libraries
import intake
from pangeo_fish.io import open_copernicus_catalog

# Open and clean reference model
cat = intake.open_catalog(catalog_url)
# true_model = open_copernicus_catalog(cat)

In [5]:
chunks=None

In [6]:
true_model = (
    cat.data(type="TEM", chunks=None)
    .to_dask()
    .rename({"thetao": "TEMP"})
    .get(["TEMP"])
    .assign_coords({"time": lambda ds: ds["time"].astype("datetime64[ns]")})).assign(
            {
                "XE": cat.data(type="SSH", chunks=chunks).to_dask().get("zos"),
                "H0": (
                    cat.data_tmp(type="mdt", chunks=chunks)
                    .to_dask()
                    .get("deptho")
                    .rename({"latitude": "lat", "longitude": "lon"})
                ),
                "mask": (
                    cat.data_tmp(type="mdt", chunks=chunks)
                    .to_dask()
                    .get("mask")
                    .rename({"latitude": "lat", "longitude": "lon"})
                ),
            }
        )

/home/aderrien/micromamba/envs/pangeo-fish/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
/home/aderrien/micromamba/envs/pangeo-fish/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


In [7]:
true_model

<xarray.Dataset> Size: 14TB
Dimensions:  (time: 41952, depth: 33, lat: 1240, lon: 958)
Coordinates:
  * depth    (depth) float32 132B 0.0 3.0 5.0 10.0 ... 2e+03 3e+03 4e+03 5e+03
  * lat      (lat) float32 5kB 46.0 46.01 46.03 46.04 ... 62.7 62.72 62.73 62.74
  * lon      (lon) float32 4kB -16.0 -15.97 -15.94 -15.91 ... 12.94 12.97 13.0
  * time     (time) datetime64[ns] 336kB 2019-01-01T01:00:00 ... 2023-10-15
Data variables:
    TEMP     (time, depth, lat, lon) float64 13TB ...
    XE       (time, lat, lon) float64 399GB ...
    H0       (lat, lon) float32 5MB ...
    mask     (depth, lat, lon) float32 157MB ...
Attributes: (12/13)
    Conventions:          CF-1.7
    contact:              servicedesk.cmems@mercator-ocean.eu
    creation_date:        2020-09-30T00:08:59Z
    credit:               E.U. Copernicus Marine Service Information (CMEMS)
    forcing_data_source:  ECMWF Global Atmospheric Model (HRES); UKMO NATL12;...
    history:              See source and creation_date attributes
    ...                   ...
    licence:              http://marine.copernicus.eu/services-portfolio/serv...
    netcdf-version-id:    netCDF-4
    product:              NORTHWESTSHELF_ANALYSIS_FORECAST_PHY_004_013
    references:           http://marine.copernicus.eu/
    source:               PS-OS 44, AMM-FOAM 1.5 km (tidal) NEMO v3.6_WAVEWAT...
    title:                hourly-instantaneous potential temperature (3D)

## **2. Loading data from this [cmems dataset](https://data.marine.copernicus.eu/product/IBI_MULTIYEAR_PHY_005_002/description)**

In [2]:
import copernicusmarine

In [3]:
name = "cmems_mod_ibi_phy_my_0.083deg"

catalogue = copernicusmarine.describe(
include_datasets=True,
contains = [name],
)

ERROR - 2024-04-11T09:14:25Z - Client version 1.0.10 is not compatible with current backend service. Please update to the latest client version.
ERROR - 2024-04-11T09:14:25Z - Client version 1.0.10 is not compatible with current backend service. Please update to the latest client version.


In [10]:
for value in catalogue['products'][0]['datasets']:
    print(value["dataset_id"])

cmems_mod_ibi_phy_my_0.083deg-2D_PT1H-m
cmems_mod_ibi_phy_my_0.083deg-3D-climatology_P1M-m
cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m
cmems_mod_ibi_phy_my_0.083deg-3D_P1M-m
cmems_mod_ibi_phy_my_0.083deg-3D_P1Y-m
cmems_mod_ibi_phy_my_0.083deg-3D_static


In [11]:
all_uri_dict = {
    "cmems_mod_ibi_phy_my_0.083deg-2D_PT1H-m": {
        "arco-geo-series": "https://s3.waw3-1.cloudferro.com/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-2D_PT1H-m_202012/timeChunked.zarr",
        "arco-time-series": "https://s3.waw3-1.cloudferro.com/mdl-arco-geo-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-2D_PT1H-m_202012/geoChunked.zarr"
    },
    "cmems_mod_ibi_phy_my_0.083deg-3D-climatology_P1M-m": {
        "arco-geo-series": "https://s3.waw3-1.cloudferro.com/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D-climatology_P1M-m_202211/timeChunked.zarr",
        "arco-time-series": "https://s3.waw3-1.cloudferro.com/mdl-arco-geo-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D-climatology_P1M-m_202211/geoChunked.zarr"
    },
    "cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m": {
        "arco-geo-series": "https://s3.waw3-1.cloudferro.com/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m_202012/timeChunked.zarr",
        "arco-time-series": "https://s3.waw3-1.cloudferro.com/mdl-arco-geo-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m_202012/geoChunked.zarr"
    },
    "cmems_mod_ibi_phy_my_0.083deg-3D_P1M-m": {
        "arco-geo-series": "https://s3.waw3-1.cloudferro.com/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_P1M-m_202012/timeChunked.zarr",
        "arco-time-series": "https://s3.waw3-1.cloudferro.com/mdl-arco-geo-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_P1M-m_202012/geoChunked.zarr"
    },
    "cmems_mod_ibi_phy_my_0.083deg-3D_P1Y-m": {
        "arco-geo-series": "https://s3.waw3-1.cloudferro.com/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_P1Y-m_202211/timeChunked.zarr",
        "arco-time-series": "https://s3.waw3-1.cloudferro.com/mdl-arco-geo-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_P1Y-m_202211/geoChunked.zarr"
    },
    "cmems_mod_ibi_phy_my_0.083deg-3D_static": {
        "static-arco": "https://s3.waw3-1.cloudferro.com:443/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_static_202012--ext--bathy/static.zarr"
    }
}


In [12]:
for cat in all_uri_dict:
    print(cat,'\n')
    for service in all_uri_dict[cat].keys():
        if not service == 'original-files':
            print(service,"\n",all_uri_dict[cat][service])
            print("\n")
            ds = xr.open_dataset(all_uri_dict[cat][service],engine="zarr")
            display(ds.data_vars)
            print('\n')

cmems_mod_ibi_phy_my_0.083deg-2D_PT1H-m 

arco-geo-series 
 https://s3.waw3-1.cloudferro.com/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-2D_PT1H-m_202012/timeChunked.zarr




Data variables:
    mlotst   (time, latitude, longitude) float64 212GB ...
    thetao   (time, latitude, longitude) float64 212GB ...
    ubar     (time, latitude, longitude) float64 212GB ...
    uo       (time, latitude, longitude) float64 212GB ...
    vbar     (time, latitude, longitude) float64 212GB ...
    vo       (time, latitude, longitude) float64 212GB ...
    zos      (time, latitude, longitude) float64 212GB ...



arco-time-series 
 https://s3.waw3-1.cloudferro.com/mdl-arco-geo-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-2D_PT1H-m_202012/geoChunked.zarr




Data variables:
    mlotst   (time, latitude, longitude) float64 212GB ...
    thetao   (time, latitude, longitude) float64 212GB ...
    ubar     (time, latitude, longitude) float64 212GB ...
    uo       (time, latitude, longitude) float64 212GB ...
    vbar     (time, latitude, longitude) float64 212GB ...
    vo       (time, latitude, longitude) float64 212GB ...
    zos      (time, latitude, longitude) float64 212GB ...



cmems_mod_ibi_phy_my_0.083deg-3D-climatology_P1M-m 

arco-geo-series 
 https://s3.waw3-1.cloudferro.com/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D-climatology_P1M-m_202211/timeChunked.zarr




...

Data variables:
    bottomT  (time, latitude, longitude) float64 24MB ...
    mlotst   (time, latitude, longitude) float64 24MB ...
    so       (time, elevation, latitude, longitude) float64 1GB ...
    thetao   (time, elevation, latitude, longitude) float64 1GB ...
    uo       (time, elevation, latitude, longitude) float64 1GB ...
    vo       (time, elevation, latitude, longitude) float64 1GB ...
    zos      (time, latitude, longitude) float64 24MB ...



arco-time-series 
 https://s3.waw3-1.cloudferro.com/mdl-arco-geo-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_P1Y-m_202211/geoChunked.zarr




Data variables:
    bottomT  (time, latitude, longitude) float64 24MB ...
    mlotst   (time, latitude, longitude) float64 24MB ...
    so       (time, elevation, latitude, longitude) float64 1GB ...
    thetao   (time, elevation, latitude, longitude) float64 1GB ...
    uo       (time, elevation, latitude, longitude) float64 1GB ...
    vo       (time, elevation, latitude, longitude) float64 1GB ...
    zos      (time, latitude, longitude) float64 24MB ...



cmems_mod_ibi_phy_my_0.083deg-3D_static 

static-arco 
 https://s3.waw3-1.cloudferro.com:443/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_static_202012--ext--bathy/static.zarr




Data variables:
    deptho       (latitude, longitude) float32 417kB ...
    deptho_lev   (latitude, longitude) float32 417kB ...
    mask_thetao  (elevation, latitude, longitude) int8 5MB ...
    mask_uo      (elevation, latitude, longitude) int8 5MB ...
    mask_vo      (elevation, latitude, longitude) int8 5MB ...

In [13]:
from pprint import pprint
import xarray as xr
import copernicusmarine as copernicusmarine

In [14]:
uris_by_key = {}  # Dictionary to store URIs by key

def get_copernicus_data(name,format='arco-geo-series' ):
    #format='arco-geo-series' or 'arco-time-series'
    catalogue = copernicusmarine.describe(
    include_datasets=True,
    contains = [name],
    )
    for value in catalogue['products'][0]['datasets']:
        dataset_id = value['dataset_id']
        if any(substring in dataset_id for substring in ['static', '2D_PT1H-m', '3D_PT1H-m','-3D_P1D-m']):
            uris = []
            for service in value['versions'][0]['parts'][0]['services']:
                service_name = service['service_type']['service_name']
                if service_name in [format,"arco-time-series","static-arco"]:
                    uris.append(service['uri'])
            uris_by_key[dataset_id] = uris
            
    
#    bbox = {"latitude": [48, 49], "longitude": [-6, -4]} 
    #.sel(latitude=slice(*bbox["latitude"]), longitude=slice(*bbox["longitude"]))
    print("thetao", uris_by_key[name+'-3D_P1D-m'][0])
    thetao=(
        xr.open_dataset(uris_by_key[name+'-3D_P1D-m'][0],
        engine='zarr')[['thetao']]
    #.sel(latitude=slice(*bbox["latitude"]), longitude=slice(*bbox["longitude"]))
    )
    print("zos", uris_by_key[name+'-3D_P1D-m'][0])
    zos=(
        xr.open_dataset(uris_by_key[name+'-3D_P1D-m'][0],
        engine='zarr').zos
         #.sel(latitude=slice(*bbox["latitude"]), longitude=slice(*bbox["longitude"]))
        )
    print("deptho", uris_by_key[name+'-3D_static'][0])
    deptho=(
        xr.open_dataset(uris_by_key[name+'-3D_static'][0]
        ,engine="zarr").deptho
        #.sel(latitude=slice(*bbox["latitude"]), longitude=slice(*bbox["longitude"]))
        )

    deptho["latitude"] = thetao["latitude"]
    mask=deptho.isnull()
    print('zos min,max value',zos.shape,zos.latitude[0:2].data,zos.latitude[-3:-1].data)
    print('deptho min,max value',deptho.shape,deptho.latitude[0:2].data,deptho.latitude[-3:-1].data)
    print('thetao min,max value',thetao.thetao.shape,thetao.latitude[0:2].data,thetao.latitude[-3:-1].data)

    

    ds=(
        thetao.rename({"thetao": "TEMP"})
        .assign(
                {
                    "XE": zos,
                    "H0": deptho,
                    "mask": mask,
                }
        )).rename({"latitude": "lat", "longitude": "lon", "elevation": "depth"})
    return ds
    

In [15]:
model = get_copernicus_data(name='cmems_mod_ibi_phy_my_0.083deg')

ERROR - 2024-04-10T15:42:17Z - Client version 1.0.10 is not compatible with current backend service. Please update to the latest client version.
ERROR - 2024-04-10T15:42:17Z - Client version 1.0.10 is not compatible with current backend service. Please update to the latest client version.
thetao https://s3.waw3-1.cloudferro.com/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m_202012/timeChunked.zarr
zos https://s3.waw3-1.cloudferro.com/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m_202012/timeChunked.zarr
deptho https://s3.waw3-1.cloudferro.com:443/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_static_202012--ext--bathy/static.zarr
zos min,max value (10589, 361, 289) [26.       26.083334] [55.833332 55.916664]
deptho min,max value (361, 289) [26.       26.083334] [55.833332 55.916664]
thetao min,max value (10589, 50, 361, 289) [26.       26.083334] [55.833332 55.916664]


In [16]:
model

<xarray.Dataset> Size: 451GB
Dimensions:  (time: 10589, depth: 50, lat: 361, lon: 289)
Coordinates:
  * depth    (depth) float32 200B -5.698e+03 -5.292e+03 ... -1.556 -0.5058
  * lat      (lat) float32 1kB 26.0 26.08 26.17 26.25 ... 55.75 55.83 55.92 56.0
  * lon      (lon) float32 1kB -19.0 -18.92 -18.83 -18.75 ... 4.833 4.917 5.0
  * time     (time) datetime64[ns] 85kB 1993-01-01 1993-01-02 ... 2021-12-28
Data variables:
    TEMP     (time, depth, lat, lon) float64 442GB ...
    XE       (time, lat, lon) float64 9GB ...
    H0       (lat, lon) float32 417kB nan nan nan nan ... 47.31 42.83 nan nan
    mask     (lat, lon) bool 104kB True True True True ... False False True True
Attributes: (12/19)
    Conventions:    CF-1.0
    bulletin_date:  2020-12-01
    bulletin_type:  Reanalysis
    contact:        mailto: servicedesk.cmems@mercator-ocean.eu
    domain_name:    IBI12
    easting:        longitude
    ...             ...
    northing:       latitude
    references:     http://marine.copernicus.eu
    source:         CMEMS IBI-MFC
    title:          CMEMS IBI REANALYSIS: DAILY PHYSICAL PRODUCTS 
    z_max:          5698.061f
    z_min:          0.50576f

In [4]:
thetao = xr.open_dataset("https://s3.waw3-1.cloudferro.com/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m_202012/timeChunked.zarr",engine="zarr")[["thetao"]]

In [5]:
thetao

<xarray.Dataset> Size: 442GB
Dimensions:    (time: 10589, elevation: 50, latitude: 361, longitude: 289)
Coordinates:
  * elevation  (elevation) float32 200B -5.698e+03 -5.292e+03 ... -1.556 -0.5058
  * latitude   (latitude) float32 1kB 26.0 26.08 26.17 ... 55.83 55.92 56.0
  * longitude  (longitude) float32 1kB -19.0 -18.92 -18.83 ... 4.833 4.917 5.0
  * time       (time) datetime64[ns] 85kB 1993-01-01 1993-01-02 ... 2021-12-28
Data variables:
    thetao     (time, elevation, latitude, longitude) float64 442GB ...
Attributes: (12/19)
    Conventions:    CF-1.0
    bulletin_date:  2020-12-01
    bulletin_type:  Reanalysis
    contact:        mailto: servicedesk.cmems@mercator-ocean.eu
    domain_name:    IBI12
    easting:        longitude
    ...             ...
    northing:       latitude
    references:     http://marine.copernicus.eu
    source:         CMEMS IBI-MFC
    title:          CMEMS IBI REANALYSIS: DAILY PHYSICAL PRODUCTS 
    z_max:          5698.061f
    z_min:          0.50576f

In [18]:
zos = xr.open_dataset("https://s3.waw3-1.cloudferro.com/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m_202012/timeChunked.zarr",engine="zarr").zos

In [12]:
deptho = xr.open_dataset("https://s3.waw3-1.cloudferro.com:443/mdl-arco-time-032/arco/IBI_MULTIYEAR_PHY_005_002/cmems_mod_ibi_phy_my_0.083deg-3D_static_202012--ext--bathy/static.zarr",engine="zarr")

In [14]:
deptho.latitude

<xarray.DataArray 'latitude' (latitude: 361)> Size: 1kB
array([26.      , 26.083334, 26.166668, ..., 55.83356 , 55.916897, 56.00023 ],
      dtype=float32)
Coordinates:
  * latitude  (latitude) float32 1kB 26.0 26.08 26.17 26.25 ... 55.83 55.92 56.0
Attributes:
    axis:           Y
    standard_name:  latitude
    step:           0.08333f
    units:          degrees_north

In [20]:
thetao.thetao

<xarray.DataArray 'thetao' (time: 10589, elevation: 50, latitude: 361,
                            longitude: 289)> Size: 442GB
[55236989050 values with dtype=float64]
Coordinates:
  * elevation  (elevation) float32 200B -5.698e+03 -5.292e+03 ... -1.556 -0.5058
  * latitude   (latitude) float32 1kB 26.0 26.08 26.17 ... 55.83 55.92 56.0
  * longitude  (longitude) float32 1kB -19.0 -18.92 -18.83 ... 4.833 4.917 5.0
  * time       (time) datetime64[ns] 85kB 1993-01-01 1993-01-02 ... 2021-12-28
Attributes:
    long_name:      Temperature
    standard_name:  sea_water_potential_temperature
    unit_long:      degrees_C
    units:          degrees_C
    valid_max:      22000
    valid_min:      -12000

In [34]:
deptho.latitude [-1] - thetao.latitude[-1] 

<xarray.DataArray 'latitude' ()> Size: 4B
array(0.00022888, dtype=float32)

### **Plotting the model data**

In [21]:
model.TEMP.isel(time=0,depth=-1).hvplot(x='lon',y='lat')

:Image   [lon,lat]   (TEMP)

In [22]:
model.H0.hvplot()

:Image   [lon,lat]   (H0)

In [23]:
model.XE.isel(time=0).hvplot()

:Image   [lon,lat]   (XE)

# **3. Adding the dynamic depth and bathymetry**
This section is under developpement

In [24]:
from pangeo_fish.io import broadcast_variables
model["depth"] = abs(model["depth"]).sortby(variables="depth")
model = model.assign(
        {
            "dynamic_depth": lambda ds: (ds["depth"] + ds["XE"]).assign_attrs(
                {"units": "m", "positive": "down"}
            ),
            "dynamic_bathymetry": lambda ds: (ds["H0"] + ds["XE"]).assign_attrs(
                {"units": "m", "positive": "down"}
            ),
        }
    ).pipe(broadcast_variables, {"lat": "latitude", "lon": "longitude"})

MemoryError: Unable to allocate 412. GiB for an array with shape (50, 10589, 361, 289) and data type float64